In [ ]:
from typing import Union, Tuple, List
from transformers import BertConfig, BertForPreTraining, BertTokenizerFast
import numpy as np
import random
import pandas as pd
from datetime import datetime, date
from tqdm.notebook import tqdm
from sklearn.metrics import log_loss, accuracy_score, precision_score, recall_score
from tqdm import tqdm
# from IPython.display import Image
from joblib import Parallel, delayed
import torch
import torch.nn as nn
from torch.nn.init import normal_
from torch.utils.data import TensorDataset, DataLoader
import torch
from transformers import BertTokenizer, BertModel
import re
# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)
from torch.utils.data.dataset import Dataset
import matplotlib.pyplot as plt
from transformers import DataCollatorForLanguageModeling
import os
from transformers import Trainer, TrainingArguments
from transformers.utils import logging
logger = logging.get_logger(__name__)
from filelock import FileLock
import time
import unicodedata
import pickle
import json
from summarizer import Summarizer,TransformerSummarizer

In [ ]:
def is_english(text):
    # Remove all punctuation and whitespace
    text = ''.join(char for char in text if unicodedata.category(char) != 'Zs' and unicodedata.category(char) != 'P')
    
    # Calculate the percentage of non-English characters
    english_chars = sum(1 for char in text if 'a' <= char.lower() <= 'z')
    non_english_chars = len(text) - english_chars
    non_english_percentage = (non_english_chars / len(text)) * 100
    
    # Return True if the text is at least 90% English characters
    return non_english_percentage <= 15

In [ ]:
with open('/home/dhkim/winery/code/feature_map/item2idx.json','r') as f: item2idx = json.load(f)
basic_info = pd.read_csv('/home/dhkim/winery/data/wine_df.csv')
basic_info['wine_id'] = basic_info['url'].map(item2idx)
basic_info = basic_info[basic_info['wine_id'].isnull() == False]

In [ ]:
review_df = pd.read_csv('/home/dhkim/winery/data/review_df_cleaned.csv',encoding = 'utf-8-sig')

In [ ]:
basic_info['wine_id'] = basic_info['wine_id'].astype('int').astype('category')
review_df['wine_id'] = review_df['wine_id'].astype('int').astype('category')

In [17]:
tqdm.pandas()
review_df['eng_or_na'] = review_df['text'].progress_apply(is_english)

100%|██████████| 1151746/1151746 [46:18<00:00, 414.45it/s]  


In [18]:
eng_review= review_df[review_df['eng_or_na']==True]

In [ ]:
GPT2_model = TransformerSummarizer(transformer_type="GPT2",transformer_model_key="gpt2-medium")

In [22]:
eng_review = pd.merge(eng_review, basic_info.loc[:,['wine_id','wine_style']], on = 'wine_id',how ='inner')

In [24]:
eng_review =eng_review[eng_review['wine_style'].isna()==False]

In [27]:
merged_reviews = eng_review.groupby('wine_style').agg({'text': ' '.join})

In [30]:
top_styles = basic_info['wine_style'].value_counts().index[:150]
top_reviews = merged_reviews.loc[merged_reviews.index.isin(top_styles)]

In [31]:
top_reviews

,text
wine_style,
Alsace Pinot Gris,"Copper, salted caramel, green apple .Full body..."
Alsace Pinot Noir,Cerises fruits rouges .Nice light red taste.Qu...
Alsace Riesling,Ingen nse. Udvandede abrikos.Vin trs bien fait...
Argentinian Cabernet Sauvignon,Bom vinho.Suave frutal.No gostei.No gostei.mui...
Argentinian Chardonnay,"Clean crisp, like a frosty winters morning. Cl..."
...,...
Spanish Toro Red,"Complex, burnt wood, bold.Good wine, do over.S..."
Upper Loire White,"Some floral tones on the nose. Mineral flint, ..."
Washington State Cabernet Sauvignon,Smooth Cab with dinner or by itself. I didnt r...


In [35]:
style_summary = {}
GPT2_model = TransformerSummarizer(transformer_type="GPT2",transformer_model_key="gpt2-medium")
for style, text in tqdm(zip(top_reviews.index, top_reviews['text'])):
    text = text[:1000000]
    style_summary[style] = ''.join(GPT2_model(text, min_length=30, max_length=100))


3it [28:38, 728.00s/it]/opt/conda/lib/python3.8/site-packages/summarizer/cluster_features.py:149: ConvergenceWarning: Number of distinct clusters (43) found smaller than n_clusters (717). Possibly due to duplicate points in X.
  model = self._get_model(k).fit(self.features)
4it [48:59, 922.81s/it]/opt/conda/lib/python3.8/site-packages/summarizer/cluster_features.py:149: ConvergenceWarning: Number of distinct clusters (614) found smaller than n_clusters (1129). Possibly due to duplicate points in X.
  model = self._get_model(k).fit(self.features)
5it [2:01:00, 2148.11s/it]

In [ ]:
with open('/home/dhkim/winery/data/wine_style_summary.json','w') as f: json.dump(style_summary, f)

In [ ]:
def keep_english_and_digits(text):
    # Remove any characters that are not English alphabets, digits, periods, or commas at the end of sentences
    clean_text = re.sub(r'[^a-zA-Z0-9\s.,]', '', text)
    return clean_text

In [ ]:
merged_reviews['text'] = merged_reviews['text'].apply(keep_english_and_digits)
merged_reviews = merged_reviews.sort_values(by = 'wine_id')
merged_reviews.to_csv('/home/dhkim/winery/data/merged_review.csv',encoding='utf-8-sig', index = False)

In [ ]:
merged_reviews = pd.read_csv('/home/dhkim/winery/data/merged_review.csv',encoding='utf-8-sig')

In [ ]:
def merge_text(data):
    return '\n'.join(data)

def parallel(func, input,  num_cpu):

    chunks = np.array_split(input, num_cpu)

    print('Parallelizing with ' +str(num_cpu)+'cores')
    with Parallel(n_jobs = num_cpu, backend="multiprocessing") as parallel:
        results = parallel(delayed(func)(chunks[i]) for i in range(num_cpu))

    for i,data in enumerate(results):
        if i == 0:
            output = data
        else:
            output += data

    return output

In [ ]:
text_data = parallel(merge_text, merged_reviews['text'],  8)

In [ ]:
file_path = '/home/dhkim/winery/data/text_data.txt'
from tqdm import tqdm
with open(file_path, 'w') as file:
    # Set the total file size for the progress bar
    total_size = len(text_data)

    # Create a tqdm progress bar
    with tqdm(total=total_size, unit='B', unit_scale=True, ncols=80, desc="Writing") as pbar:
        # Write the text data to the file in chunks
        for chunk in text_data:
            file.write(chunk)
            pbar.update(len(chunk))

In [ ]:
dataset = []
total_lines = len(text_data)
with tqdm(total=total_lines, ncols=80, desc="Processing") as pbar:
    for text in text_data:
        text = text.strip()
        if text:
            dataset.append(text)
        pbar.update(1)

# Step 2: Tokenization
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')


In [ ]:
from transformers import BertConfig, BertForPreTraining

config = BertConfig(    
    vocab_size=tokenizer.vocab_size,
    hidden_size=512,
    num_hidden_layers=8,    # layer num
    num_attention_heads=8,    # transformer attention head number
    intermediate_size=3072,   # transformer 내에 있는 feed-forward network의 dimension size
    hidden_act="gelu",
    hidden_dropout_prob=0.1,
    attention_probs_dropout_prob=0.1,
    max_position_embeddings=128,    # embedding size 최대 몇 token까지 input으로 사용할 것인지 지정
    pad_token_id=0,
    position_embedding_type="absolute"
)

model = BertForPreTraining(config=config)
model.num_parameters()

In [ ]:
from transformers.tokenization_utils import PreTrainedTokenizer
class TextDatasetForNextSentencePrediction(Dataset):

    def __init__(
        self,
        tokenizer: PreTrainedTokenizer,
        file_path: str,
        block_size: int,
        overwrite_cache=False,
        short_seq_probability=0.1,
        nsp_probability=0.5,
    ):
        # 여기 부분은 학습 데이터를 caching하는 부분입니다 :-)
        assert os.path.isfile(file_path), f"Input file path {file_path} not found"

        self.block_size = block_size - tokenizer.num_special_tokens_to_add(pair=True)
        self.short_seq_probability = short_seq_probability
        self.nsp_probability = nsp_probability

        directory, filename = os.path.split(file_path)
        cached_features_file = os.path.join(
            directory,
            "cached_nsp_{}_{}_{}".format(
                tokenizer.__class__.__name__,
                str(block_size),
                filename,
            ),
        )

        self.tokenizer = tokenizer

        lock_path = cached_features_file + ".lock"

        # Input file format:
        # (1) One sentence per line. These should ideally be actual sentences, not
        # entire paragraphs or arbitrary spans of text. (Because we use the
        # sentence boundaries for the "next sentence prediction" task).
        # (2) Blank lines between documents. Document boundaries are needed so
        # that the "next sentence prediction" task doesn't span between documents.
        #
        # Example:
        # I am very happy.
        # Here is the second sentence.
        #
        # A new document.

        with FileLock(lock_path):
            if os.path.exists(cached_features_file) and not overwrite_cache:
                start = time.time()
                with open(cached_features_file, "rb") as handle:
                    self.examples = pickle.load(handle)
                logger.info(
                    f"Loading features from cached file {cached_features_file} [took %.3f s]", time.time() - start
                )
            else:
                logger.info(f"Creating features from dataset file at {directory}")
                # 여기서부터 본격적으로 dataset을 만듭니다.
                self.documents = [[]]
                with open(file_path, encoding="utf-8") as f:
                    while True: # 일단 문장을 읽고
                        line = f.readline()
                        if not line:
                            break
                        line = line.strip()

                        # 이중 띄어쓰기가 발견된다면, 나왔던 문장들을 모아 하나의 문서로 묶어버립니다.
                        # 즉, 문단 단위로 데이터를 저장합니다.
                        if not line and len(self.documents[-1]) != 0:
                            self.documents.append([])
                        tokens = tokenizer.tokenize(line)
                        tokens = tokenizer.convert_tokens_to_ids(tokens)
                        if tokens:
                            self.documents[-1].append(tokens)
                # 이제 코퍼스 전체를 읽고, 문서 데이터를 생성했습니다! :-)
                logger.info(f"Creating examples from {len(self.documents)} documents.")
                self.examples = []
                # 본격적으로 학습을 위한 데이터로 변형시켜볼까요?
                for doc_index, document in enumerate(self.documents):
                    self.create_examples_from_document(document, doc_index) # 함수로 가봅시다.

                start = time.time()
                with open(cached_features_file, "wb") as handle:
                    pickle.dump(self.examples, handle, protocol=pickle.HIGHEST_PROTOCOL)
                logger.info(
                    "Saving features into cached file %s [took %.3f s]", cached_features_file, time.time() - start
                )

    def create_examples_from_document(self, document: List[List[int]], doc_index: int):
        """Creates examples for a single document."""
        # 문장의 앞, 뒤에 [CLS], [SEP] token이 부착되기 때문에, 내가 지정한 size에서 2 만큼 빼줍니다.
        # 예를 들어 128 token 만큼만 학습 가능한 model을 선언했다면, 학습 데이터로부터는 최대 126 token만 가져오게 됩니다.
        max_num_tokens = self.block_size - self.tokenizer.num_special_tokens_to_add(pair=True)

        # We *usually* want to fill up the entire sequence since we are padding
        # to `block_size` anyways, so short sequences are generally wasted
        # computation. However, we *sometimes*
        # (i.e., short_seq_prob == 0.1 == 10% of the time) want to use shorter
        # sequences to minimize the mismatch between pretraining and fine-tuning.
        # The `target_seq_length` is just a rough target however, whereas
        # `block_size` is a hard limit.

        # 여기가 재밌는 부분인데요!
        # 위에서 설명했듯이, 학습 데이터는 126 token(128-2)을 채워서 만들어지는게 목적입니다.
        # 하지만 나중에 BERT를 사용할 때, 126 token 이내의 짧은 문장을 테스트하는 경우도 분명 많을 것입니다 :-)
        # 그래서 short_seq_probability 만큼의 데이터에서는 2-126 사이의 random 값으로 학습 데이터를 만들게 됩니다.
        target_seq_length = max_num_tokens
        if random.random() < self.short_seq_probability:
            target_seq_length = random.randint(2, max_num_tokens)

        current_chunk = []  # a buffer stored current working segments
        current_length = 0
        i = 0

        # 데이터 구축의 단위는 document 입니다
        # 이 때, 무조건 문장_1[SEP]문장_2 이렇게 만들어지는 것이 아니라,
        # 126 token을 꽉 채울 수 있게 문장_1+문장_2[SEP]문장_3+문장_4 형태로 만들어질 수 있습니다.
        while i < len(document):
            segment = document[i]
            current_chunk.append(segment)
            current_length += len(segment)
            if i == len(document) - 1 or current_length >= target_seq_length:
                if current_chunk:
                    # `a_end` is how many segments from `current_chunk` go into the `A`
                    # (first) sentence.
                    a_end = 1
                    # 여기서 문장_1+문장_2 가 이루어졌을 때, 길이를 random하게 짤라버립니다 :-)
                    if len(current_chunk) >= 2:
                        a_end = random.randint(1, len(current_chunk) - 1)
                    tokens_a = []
                    for j in range(a_end):
                        tokens_a.extend(current_chunk[j])
                    # 이제 [SEP] 뒷 부분인 segmentB를 살펴볼까요?
                    tokens_b = []
                    # 50%의 확률로 랜덤하게 다른 문장을 선택하거나, 다음 문장을 학습데이터로 만듭니다.
                    if len(current_chunk) == 1 or random.random() < self.nsp_probability:
                        is_random_next = True
                        target_b_length = target_seq_length - len(tokens_a)

                        # This should rarely go for more than one iteration for large
                        # corpora. However, just to be careful, we try to make sure that
                        # the random document is not the same as the document
                        # we're processing.
                        for _ in range(10):
                            random_document_index = random.randint(0, len(self.documents) - 1)
                            if random_document_index != doc_index:
                                break
                        # 여기서 랜덤하게 선택합니다 :-)
                        random_document = self.documents[random_document_index]
                        random_start = random.randint(0, len(random_document) - 1)
                        for j in range(random_start, len(random_document)):
                            tokens_b.extend(random_document[j])
                            if len(tokens_b) >= target_b_length:
                                break
                        # We didn't actually use these segments so we "put them back" so
                        # they don't go to waste.
                        num_unused_segments = len(current_chunk) - a_end
                        i -= num_unused_segments
                    # Actual next
                    else:
                        is_random_next = False
                        for j in range(a_end, len(current_chunk)):
                            tokens_b.extend(current_chunk[j])

                    # 이제 126 token을 넘는다면 truncation을 해야합니다.
                    # 이 때, 126 token 이내로 들어온다면 행위를 멈추고,
                    # 만약 126 token을 넘는다면, segmentA와 segmentB에서 랜덤하게 하나씩 제거합니다.
                    def truncate_seq_pair(tokens_a, tokens_b, max_num_tokens):
                        """Truncates a pair of sequences to a maximum sequence length."""
                        while True:
                            total_length = len(tokens_a) + len(tokens_b)
                            if total_length <= max_num_tokens:
                                break
                            trunc_tokens = tokens_a if len(tokens_a) > len(tokens_b) else tokens_b
                            assert len(trunc_tokens) >= 1
                            # We want to sometimes truncate from the front and sometimes from the
                            # back to add more randomness and avoid biases.
                            if random.random() < 0.5:
                                del trunc_tokens[0]
                            else:
                                trunc_tokens.pop()

                    truncate_seq_pair(tokens_a, tokens_b, max_num_tokens)

                    assert len(tokens_a) >= 1
                    assert len(tokens_b) >= 1

                    # add special tokens
                    input_ids = self.tokenizer.build_inputs_with_special_tokens(tokens_a, tokens_b)
                    # add token type ids, 0 for sentence a, 1 for sentence b
                    token_type_ids = self.tokenizer.create_token_type_ids_from_sequences(tokens_a, tokens_b)
                    
                    # 드디어 아래 항목에 대한 데이터셋이 만들어졌습니다! :-)
                    # 즉, segmentA[SEP]segmentB, [0, 0, .., 0, 1, 1, ..., 1], NSP 데이터가 만들어진 것입니다 :-)
                    # 그럼 다음은.. 이 데이터에 [MASK] 를 씌워야겠죠?
                    example = {
                        "input_ids": torch.tensor(input_ids, dtype=torch.long),
                        "token_type_ids": torch.tensor(token_type_ids, dtype=torch.long),
                        "next_sentence_label": torch.tensor(1 if is_random_next else 0, dtype=torch.long),
                    }

                    self.examples.append(example)

                current_chunk = []
                current_length = 0

            i += 1

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i):
        return self.examples[i]

In [ ]:
from tokenizers import BertWordPieceTokenizer
wp_tokenizer = BertWordPieceTokenizer(
    clean_text=True,  
    strip_accents=False,    
    lowercase=True,
)
wp_tokenizer.train(
    files="/home/dhkim/winery/data/text_data.txt",
    vocab_size=40000,  
    min_frequency=3,
    show_progress=True,
    special_tokens=["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"],
    wordpieces_prefix="##"
)

In [ ]:
wp_tokenizer.save_model("/home/dhkim/winery/code/models", "review_tokenizer")

In [ ]:

tokenizer = BertTokenizerFast(
    vocab_file='/home/dhkim/winery/code//text/models/review_tokenizer-vocab.txt',
    max_len=128,
    do_lower_case=True,
    )

In [ ]:

dataset = TextDatasetForNextSentencePrediction(
    tokenizer=tokenizer,
    file_path='/home/dhkim/winery/data/text_data.txt',
    block_size=128,
    overwrite_cache=False,
    short_seq_probability=0.2,
    nsp_probability=0.5,
)

data_collator = DataCollatorForLanguageModeling(    # [MASK] 를 씌우는 것은 저희가 구현하지 않아도 됩니다! :-)
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [ ]:
training_args = TrainingArguments(
    output_dir='/home/dhkim/winery/code/models',
    overwrite_output_dir=True,
    num_train_epochs=1000,
    per_gpu_train_batch_size=32,
    save_steps=1000,
    save_total_limit=2,
    logging_steps=100
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
)

In [ ]:
trainer.train() # wiki 전체 데이터로 학습 시, 1 epoch에 9시간 정도 소요됩니다!! 

In [ ]:
trainer.save_model('/home/dhkim/winery/code/models/model_output')

In [ ]:
trainer.model


In [ ]:
model = BertModel.from_pretrained('/home/dhkim/winery/code/text/models/model_output')

In [ ]:
def get_embedding(df, model):
    review_vectors = {}

    with torch.no_grad():

        for i in tqdm(range(len(df))):
            reviews = df['text'][i].split('.')
            id = df['wine_id'][i]
            review_vector = []

            for text in tqdm(reviews):
                try:
                    encoded_input = tokenizer.encode_plus(
                        text, 
                        truncation = True,
                        add_special_tokens=True, 
                        return_tensors='pt')
                    model_output = model(**encoded_input)
                    embeddings = model_output.last_hidden_state
                    sentence_embedding = torch.mean(embeddings[0], dim=0)
                    review_vector.append(sentence_embedding)

                    del embeddings
                    del model_output
                    del encoded_input
                    gc.collect()
                    
                except: 1

            del review_vector
            del sentence_embedding
            gc.collect()
            mean_vector = torch.mean(torch.stack(review_vector), dim=0).numpy()
            review_vectors[id] = mean_vector
        
    return review_vectors
                    


In [ ]:
#review_vectors = parallel_embedding(merged_reviews, model, 8)
review_vectors = get_embedding(merged_reviews, model)